In [1]:
pip install plaid-python


  Using cached plaid_python-24.0.0-py3-none-any.whl
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached nulltype-2.3.1-py2.py3-none-any.whl.metadata (10 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
Using cached nulltype-2.3.1-py2.py3-none-any.whl (11 kB)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\nmessaoudi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [44]:
pip install django

   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB 660.6 kB/s eta 0:00:13
   - -------------------------------------- 0.2/8.2 MB 2.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/8.2 MB 8.1 MB/s eta 0:00:01
   ----------- ---------------------------- 2.4/8.2 MB 15.0 MB/s eta 0:00:01
   -------------------- ------------------- 4.1/8.2 MB 20.1 MB/s eta 0:00:01
   -------------------------- ------------- 5.5/8.2 MB 21.9 MB/s eta 0:00:01
   ----------------------------------- ---- 7.4/8.2 MB 24.7 MB/s eta 0:00:01
   ---------------------------------------  8.2/8.2 MB 26.1 MB/s eta 0:00:01
   ---------------------------------------- 8.2/8.2 MB 23.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.2 kB ? eta -:--:--
   ---------------------------------------- 44.2/44.2 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\nmessaoudi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [60]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\nmessaoudi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [66]:
from plaid.api import plaid_api
from plaid.model.plaid_error import PlaidError
from plaid.model.products import Products
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.country_code import CountryCode
from plaid.model.link_token_get_request import LinkTokenGetRequest
from plaid.model.accounts_get_request import AccountsGetRequest
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.configuration import Configuration
from plaid.api_client import ApiClient
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
import os

import json
import datetime
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file


In [67]:
load_dotenv() 
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SECRET")
PLAID_ENV = os.getenv("PLAID_ENV")

Fake data from sandbox

In [76]:
# Step 1: Initialize Plaid client configuration


configuration = Configuration(
    host=PLAID_ENV,
    api_key={
        'clientId': PLAID_CLIENT_ID,
        'secret': PLAID_SECRET,
    }
)

api_client = ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

request = LinkTokenCreateRequest(
    products=[Products('auth'), Products('transactions')],
    client_name="mycompany",
    country_codes=[CountryCode('FR')],
    language='fr',
    user={'client_user_id': 'user_12345'}
)

response = client.link_token_create(request)
link_token = response['link_token']

In [77]:
# Generate public token
pt_request = SandboxPublicTokenCreateRequest(
    institution_id='ins_117650',
    initial_products=[Products('transactions')]
)
pt_response = client.sandbox_public_token_create(pt_request)


In [78]:
# The generated public_token can now be
# exchanged for an access_token
exchange_request = ItemPublicTokenExchangeRequest(
    public_token=pt_response['public_token']
)
exchange_response = client.item_public_token_exchange(exchange_request)
access_token = exchange_response.access_token

In [79]:
# Retrieve account data using access token
request = AccountsGetRequest(
    access_token=access_token
)

try:
    accounts_response = client.accounts_get(request)
    accounts = accounts_response['accounts']
    accounts_list = [account.to_dict() for account in accounts]  # Convert to dictionary
    print(json.dumps(accounts_list, indent=2))
except Exception as e:
    print("Error retrieving accounts:", e)
    if hasattr(e, 'body'):
        try:
            error_response = json.loads(e.body)
            print(json.dumps(error_response, indent=2))
        except json.JSONDecodeError:
            print("Error decoding JSON response")


[
  {
    "account_id": "R6dG1D6MpJhw7byjNmWmfj8a66EgjbCao5vmZ",
    "balances": {
      "available": 100.0,
      "current": 110.0,
      "limit": null,
      "iso_currency_code": "GBP",
      "unofficial_currency_code": null
    },
    "mask": "0000",
    "name": "Plaid Current Account",
    "official_name": "Plaid Standard Current Account",
    "type": "depository",
    "subtype": "checking",
    "persistent_account_id": "8cfb8beb89b774ee43b090625f0d61d0814322b43bff984eaf60386e"
  },
  {
    "account_id": "65QvBE56GJunpj7E6wVwS7oMBBDZ7ks8PnoVk",
    "balances": {
      "available": 200.0,
      "current": 210.0,
      "limit": null,
      "iso_currency_code": "GBP",
      "unofficial_currency_code": null
    },
    "mask": "1111",
    "name": "Plaid Saving",
    "official_name": "Plaid Standard Interest Saving",
    "type": "depository",
    "subtype": "savings",
    "persistent_account_id": "211a4e5d8361a3afb7a3886362198c7306e00a313b5aa944c20d34b6"
  },
  {
    "account_id": "X67rw

In [81]:
# Retrieve transactions data using access token

def serialize_response(obj):
    if isinstance(obj, (datetime.date, datetime.datetime)):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [serialize_response(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_response(value) for key, value in obj.items()}
    else:
        return obj

start_date = datetime.datetime.now() - datetime.timedelta(days=30)
end_date = datetime.datetime.now()

request = TransactionsGetRequest(
    access_token=access_token,
    start_date=start_date.date(),
    end_date=end_date.date()
)

try:
    transactions_response = client.transactions_get(request)
    transactions = transactions_response['transactions']
    
    # Convert to dictionary and handle serialization
    transactions_list = [serialize_response(transaction.to_dict()) for transaction in transactions]
    
    print(json.dumps(transactions_list, indent=2))
except Exception as e:
    print("Error retrieving transactions:", e)
    if hasattr(e, 'body'):
        try:
            error_response = json.loads(e.body)
            print(json.dumps(error_response, indent=2))
        except json.JSONDecodeError:
            print("Error decoding JSON response")


[
  {
    "account_id": "X67rwW6GAphRLX5DZ6e6igVjxxPpgLUbv8Ww8",
    "account_owner": null,
    "amount": 500.0,
    "authorized_date": null,
    "authorized_datetime": null,
    "category": [
      "Shops",
      "Clothing and Accessories"
    ],
    "category_id": "19012000",
    "check_number": null,
    "counterparties": [
      {
        "name": "Burberry",
        "type": "merchant",
        "website": null,
        "logo_url": null,
        "confidence_level": "VERY_HIGH",
        "entity_id": null,
        "phone_number": null
      }
    ],
    "date": "2024-08-02",
    "datetime": null,
    "iso_currency_code": "GBP",
    "location": {
      "address": null,
      "city": null,
      "region": null,
      "postal_code": null,
      "country": null,
      "lat": null,
      "lon": null,
      "store_number": null
    },
    "logo_url": null,
    "merchant_entity_id": null,
    "merchant_name": "Burberry",
    "name": "Burberry",
    "payment_channel": "in store",
    "payment_

Real data from dev environment

Create ML model to categorize transactions

Compare with plaid categorization